# Multi-Agent Actor Critic

- 앞선 알고리즘의 경우 하나의 agent의 학습을 진행했었다. 하지만, 현실 세계에는 여러 agent의 상호작용이 필요한 경우가 존재한다.
- 위 문제를 해결하기 위해서 Multi-Agent actor critic 알고리즘이 제안되었다.

- 따라서, 여러 agent가 참여하여 특정 목적을 달성하도록 학습시키는 강화학습이다.
- DDPG를 Multi-agent 환경에 적용한 것이다.

## 가정

- actor critic policy gradient를 기반으로 하고,
- actor는 다른 agent의 policy를 알지 못한다.
- agent는 observation가능한 정보만 사용해야한다. 즉, 모든 환경을 알 수는 없다.
- critic에서 action-value function은 각각의 agent마다 가진 policy를 이용하여 계산하지만, 다른 agent의 action도 반영된다.
- 학습하는 agent는 다른 agent의 policy 변화에 능동적으로 반응하여 최적의 action을 찾아낸다.

## Multi-agent MDP, DDPG

- MDP에서는 state, action, policy와 space를 명확히 알고있어야한다.
- 이전 알고리즘들은 single agent MDP문제였다.
- 하지만 multi agent DDPG는
  - 모든 agent의 행동이 포함된 state space, \\(\mathcal{S}\\)
  - 각 agent의 서로 다른 observation space, action space, \\(\mathcal{O}\\), \\(\mathcal{A}\\)
  - 각각의 agent마다 observation과 action space의 곱으로 표현되는 stochastic policy space, \\(\mathcal{O} \times \mathcal{A}\\)
  - state space와 모든 agent의 action space의 곱으로 표현되는 Transition space \\(\mathcal{T}\\), state와 action의 상호작용에 대한 정보가 들어있음.
  - 각 agent는 reward space도 가진다.

## Network

- Multi-agent DDPG에서는
  - critic, DQN에서 사용하는 Q-value network를 사용한다.
  - actor, policy gradient method를 사용한다.
  - 또한, 각 agent들이 상호보완적, 적대적으로 학습의 방향성을 주기 위해서 centralized critic을 사용한다.

![default](https://user-images.githubusercontent.com/22078438/50497544-ddcd8680-0a7b-11e9-9448-f808f7e0f45a.PNG)


- Policy gradient method는 discounted cumulative reward, objective function \\(J\\)를 최대화하는 것이 목표이고 Multi-agent DDPG에서 아래와 같이 objective function을 나타낼 수 있다.<br><br>
$$ \triangledown_{\theta_i}J(\mu_i) = \mathbb{E}_{X, a \sim D} \left[ \triangledown_{\theta_i} \mu_i(a_i|o_i) \triangledown_{a_i} Q^{\mu}_i(X, a_1, ..., a_N)|_{a_i=\mu_i(o_i)} \right] $$

- 위 식을 보면 Q-value의 state와 모든 agent의 action을 잘 나타내고 있다. 또한, 학습과정에서 모든 agent의 action이 고려된다는 것을 의미한다.
- 따라서, centralized action-value function이라 하는데 모든 agent의 action \\(a_1, ... , a_N\\)과 state 정보 \\(X = (O_1, ..., O_N)\\)을 입력으로 해서 agent \\(i\\)의 action-value function을 구한다.
- 마지막으로, replay buffer \\(D\\)는 tuple의 형태로 저장한다.

- centralized action-value function \\(Q^{\mu_i}\\)는 아래 식으로 update한다.<br><br>
$$ \mathcal{L}(\theta_i) = \mathbb{E}_{x, a, r, x'} \left[{( Q^{\mu}_i(x, a_1, ..., a_N) - y )}^2 \right] $$

$$ y = r_i + \gamma Q^{\mu'}_i(x', {a'}_1, ... ,{a'}_N)|_{{a'}_j = {\mu'}_j(o_j)} $$

- 위 식에서 \\(\mu' = \{ \mu_{\theta'_1}, ..., \mu_{\theta'_N} \}\\)이고, target policy의 집합이다. 이 식은 next state에서 다른 agent의 action을 알아야 update가 가능하다.

## agent policy approximation

- agent 집합이 개별적 학습을 통해서 policy를 update시키도록 해야하고, 이 policy가 반영된 action에 의한 Q-value가 점점 target에 가까워져야 한다. 하지만 이것은 local optimum에 빠질수 있고,
- 따라서 다른 agent의 policy를 추론하는 방법을 설명한다.

- agent \\(i\\)는 추가로 다른 agnet \\(j\\)의 true policy \\(\mu_j\\)에 대응하는 approximation policy \\(\hat{\mu}^j_i\\)를 계속 추론한다. 다른 agent들의 approximation policy는 action의 log probability를 최대화 하는 방향으로 추론하였으며, entropy regularizer(H)를 사용하였다.<br><br>
$$ \mathcal{L}(\phi^j_i) = -\mathbb{E}_{o_j, a_j} \left[ log \hat{\mu}^j_i(a_j|o_j) + \lambda H(\hat{\mu}^j_i) \right] $$

- \\(\phi\\)는 policy approximator의 parameter를 의미한다. 따라서 approximation policy를 사용하여 위 식의 y는 \\(\hat{y}\\)로 대체할 수 있다.<br><br>

$$ \hat{y} = r_i + \gamma Q^{\mu'}_i (x', \hat{\mu}^{'1}_i(o_1), ..., \hat{\mu}^{'N}_i(o_N)) $$

## Agent policy ensembles

- DQN의 안정성을 증가시키는 방법을 여기서 사용하였지만 multi-agent가 존재하는 환경에서 안정적으로 수렴한다는 것은 매우 어려운 일이다. 따라서 \\(K\\)개의 서로다른 sub-policy로 이루어진 policy의 ensemble을 학습시키는 방법을 도입하였고, 매 episode마다 각 agent가 실행시킬 sub-policy를 랜덤으로 선택해서 사용한다.

- 따라서, \\(\mu_i\\)는 agent \\(i\\)가 가지는 하나의 policy가 아니라 \\(K\\)개의 sub-policy로 이루어진 ensemble을 나타낸다. ensemble에서 선택된 policy \\(k\\)는 \\(\mu^{(k)}_i\\)로 표기하고 아래의 ensemble objective function을 최대화 하도록 학습한다.<br><br>
$$ J_e(\mu_i) = \mathbb{E}_{k \sim unif(1, K), s \sim p^\mu, a \sim \mu_i^{(k)}} \left [ R_i(s, a) \right] $$

- episode마다 다른 sub-policy가 실행되기 때문에 agent \\(i\\)의 각 sub-policy \\(\mu^{(k)}_i\\)에 대해 replay buffer \\(D^{(k)}_i\\)를 사용하고, 위 ensemble obejctive의 gradient를 구하면 아래와 같다.<br><br>
$$ \triangledown_{\theta^{(k)}_i} J_e(\mu_i) = \frac{1}{K} \mathbb{E}_{x,a \sim D^{(k)}_i} \left[ \triangledown_{\theta^{(k)}_i}(a_i|o_i) \triangledown_{a_i} Q^{\mu}_i(x, a_1, ..., a_N)|_{a_i=\mu_i^{(k)}(o_i)} \right] $$

- 결론적으로, policy ensemble에서 policy를 모아둔 policy basket을 사용하겠다는 것이고 episode마다 다른 policy를 사용하게 되면서 학습에 사용되는 replay buffer도 다르게 사용한다.